From episcanpy use epi.tl.find_genes to find the peaks

# File opening

In [3]:
#edit based on your file path

file_path1 = "/Users/gajaj/Downloads/H_Kidney_Cancer_Chromium_Nuc_Iso_vs_SaltyEZ_vs_ComplexTissueDP_raw_feature_bc_matrix.tar.gz"
output_dir1 = "/Users/gajaj/OneDrive/Documents/TUM/computational_single_cell/Gene-expression-changes-from-CNV/preprocessing/Multiome"

In [4]:
with tarfile.open(file_path1, "r") as tar:
    tar.extractall(path=output_dir1)
    print("Files extracted successfully")

Files extracted successfully


In [5]:
#edit your file paths
input_file_barcodes1 = "Multiome/raw_feature_bc_matrix/barcodes.tsv.gz"
input_file_features1 = "Multiome/raw_feature_bc_matrix/features.tsv.gz"
input_file_matrix1 = "Multiome/raw_feature_bc_matrix/matrix.mtx.gz"

In [6]:
#edit your file paths
output_file_barcodes1 = "Multiome/raw_feature_bc_matrix/barcodes.tsv"
output_file_features1 = "Multiome/raw_feature_bc_matrix/features.tsv"
output_file_matrix1 = "Multiome/raw_feature_bc_matrix/matrix.mtx"


In [7]:
import gzip

with gzip.open(input_file_barcodes1, "rt") as gz_file:  
    with open(output_file_barcodes1, "w") as out_file:
        out_file.write(gz_file.read())

In [8]:
with gzip.open(input_file_features1, "rt") as gz_file:  
    with open(output_file_features1, "w") as out_file:
        out_file.write(gz_file.read())

In [9]:
with gzip.open(input_file_matrix1, "rt") as gz_file:  
    with open(output_file_matrix1, "w") as out_file:
        out_file.write(gz_file.read())

# Multiome

In [2]:
!pip install pyranges

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 13.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/772.9 kB ? eta -:--:--
   --------------------------------------- 772.9/772.9 kB 16.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 10.2 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import pandas as pd
import tarfile
import os
import pyranges as pr

In [2]:
import muon as mu

Read the multiome data

In [3]:
data_dir ='Multiome'
mdata = mu.read_10x_h5(os.path.join(data_dir, 'H_Kidney_Cancer_Chromium_Nuc_Iso_vs_SaltyEZ_vs_ComplexTissueDP_filtered_feature_bc_matrix.h5'))

c:\Users\gajaj\anaconda3\envs\preprocessing\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Added `interval` annotation for features from Multiome\H_Kidney_Cancer_Chromium_Nuc_Iso_vs_SaltyEZ_vs_ComplexTissueDP_filtered_feature_bc_matrix.h5


c:\Users\gajaj\anaconda3\envs\preprocessing\lib\site-packages\mudata\_core\mudata.py:477: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


In [4]:
mdata.var_names_make_unique()
mdata

MuData object with n_obs × n_vars = 22772 × 103751
  var:	'gene_ids', 'feature_types', 'genome', 'interval'
  2 modalities
    rna:	22772 x 36601
      var:	'gene_ids', 'feature_types', 'genome', 'interval'
    atac:	22772 x 67150
      var:	'gene_ids', 'feature_types', 'genome', 'interval'

In [5]:
atac = mdata.mod['atac']
atac  # an AnnData object

AnnData object with n_obs × n_vars = 22772 × 67150
    var: 'gene_ids', 'feature_types', 'genome', 'interval'

In [6]:
atac.var


,gene_ids,feature_types,genome,interval
chr1:9855-10676,chr1:9855-10676,Peaks,GRCh38,chr1:9855-10676
chr1:181027-181582,chr1:181027-181582,Peaks,GRCh38,chr1:181027-181582
chr1:629494-630387,chr1:629494-630387,Peaks,GRCh38,chr1:629494-630387
chr1:633523-634601,chr1:633523-634601,Peaks,GRCh38,chr1:633523-634601
chr1:778319-779215,chr1:778319-779215,Peaks,GRCh38,chr1:778319-779215
...,...,...,...,...
KI270713.1:21530-22452,KI270713.1:21530-22452,Peaks,GRCh38,KI270713.1:21530-22452
KI270713.1:26160-26918,KI270713.1:26160-26918,Peaks,GRCh38,KI270713.1:26160-26918
KI270713.1:28410-29017,KI270713.1:28410-29017,Peaks,GRCh38,KI270713.1:28410-29017
KI270713.1:32755-33461,KI270713.1:32755-33461,Peaks,GRCh38,KI270713.1:32755-33461


In [7]:
atac.obs

""
AAACAGCCAAACAACA-4
AAACAGCCAAACGCGA-2
AAACAGCCAAGGACCA-3
AAACAGCCAAGGTACG-4
AAACAGCCAATGAATG-3
...
TTTGTTGGTGGCTTCC-1
TTTGTTGGTGTAACCA-3
TTTGTTGGTTAAGTGT-2
TTTGTTGGTTAGCTGA-2


In [89]:
atac.X.todense().max()

794.0

Load the dataset of gene positions created in `get_gene_position.r`

In [8]:
# Load gene regions dataset
genes_df = pd.read_csv("Multiome/gene_positions.csv")

In [9]:
genes_df.columns = ["seqnames", "start", "end", "gene_id"]
genes_df

,seqnames,start,end,gene_id
0,1,3069168,3438621,ENSG00000142611
1,1,5301928,5307394,ENSG00000284616
2,1,2403964,2413797,ENSG00000157911
3,1,5492978,5494674,ENSG00000260972
4,1,10054445,10054781,ENSG00000224340
...,...,...,...,...
78927,KI270717.1,15119,17798,ENSG00000307722
78928,KI270717.1,24947,37269,ENSG00000310401
78929,KI270720.1,3633,31375,ENSG00000302039
78930,KI270718.1,6913,19848,ENSG00000309831


Get chromosomes and regions from the atac

In [10]:
atac.var[['chrom', 'start', 'end']] = atac.var['interval'].str.split(':|-', expand=True)
peaks = atac.var[['chrom', 'start', 'end']].copy()
peaks

,chrom,start,end
chr1:9855-10676,chr1,9855,10676
chr1:181027-181582,chr1,181027,181582
chr1:629494-630387,chr1,629494,630387
chr1:633523-634601,chr1,633523,634601
chr1:778319-779215,chr1,778319,779215
...,...,...,...
KI270713.1:21530-22452,KI270713.1,21530,22452
KI270713.1:26160-26918,KI270713.1,26160,26918
KI270713.1:28410-29017,KI270713.1,28410,29017
KI270713.1:32755-33461,KI270713.1,32755,33461


In [11]:
# Remove 'chr' prefix if it exists
peaks["chrom"] = peaks["chrom"].str.replace("^chr", "", regex=True)

# Display the modified DataFrame
peaks


,chrom,start,end
chr1:9855-10676,1,9855,10676
chr1:181027-181582,1,181027,181582
chr1:629494-630387,1,629494,630387
chr1:633523-634601,1,633523,634601
chr1:778319-779215,1,778319,779215
...,...,...,...
KI270713.1:21530-22452,KI270713.1,21530,22452
KI270713.1:26160-26918,KI270713.1,26160,26918
KI270713.1:28410-29017,KI270713.1,28410,29017
KI270713.1:32755-33461,KI270713.1,32755,33461


In [12]:
row = peaks[peaks['end'] == 30267]
print(row)

Empty DataFrame
Columns: [chrom, start, end]
Index: []


In [13]:
genes_df = genes_df.rename(columns={
    "seqnames": "Chromosome",  # Rename seqnames to Chromosome
    "start": "Start",          # Rename start to Start
    "end": "End"               # Rename end to End
})
genes_df

,Chromosome,Start,End,gene_id
0,1,3069168,3438621,ENSG00000142611
1,1,5301928,5307394,ENSG00000284616
2,1,2403964,2413797,ENSG00000157911
3,1,5492978,5494674,ENSG00000260972
4,1,10054445,10054781,ENSG00000224340
...,...,...,...,...
78927,KI270717.1,15119,17798,ENSG00000307722
78928,KI270717.1,24947,37269,ENSG00000310401
78929,KI270720.1,3633,31375,ENSG00000302039
78930,KI270718.1,6913,19848,ENSG00000309831


In [14]:
genes_pr = pr.PyRanges(genes_df)
genes_pr

,Chromosome,Start,End,gene_id
0,1,3069168,3438621,ENSG00000142611
1,1,5301928,5307394,ENSG00000284616
2,1,2403964,2413797,ENSG00000157911
3,1,5492978,5494674,ENSG00000260972
4,1,10054445,10054781,ENSG00000224340
...,...,...,...,...
78927,Y,9898786,9908137,ENSG00000309435
78928,Y,6266593,6269275,ENSG00000238235
78929,Y,6266602,6275213,ENSG00000296934
78930,Y,6328052,6343323,ENSG00000223744


In [15]:
peaks = peaks.rename(columns={"chrom": "Chromosome", "start": "Start", "end": "End"})
peaks

,Chromosome,Start,End
chr1:9855-10676,1,9855,10676
chr1:181027-181582,1,181027,181582
chr1:629494-630387,1,629494,630387
chr1:633523-634601,1,633523,634601
chr1:778319-779215,1,778319,779215
...,...,...,...
KI270713.1:21530-22452,KI270713.1,21530,22452
KI270713.1:26160-26918,KI270713.1,26160,26918
KI270713.1:28410-29017,KI270713.1,28410,29017
KI270713.1:32755-33461,KI270713.1,32755,33461


In [16]:
peaks_pr = pr.PyRanges(peaks)
peaks_pr

,Chromosome,Start,End
0,1,9855,10676
1,1,181027,181582
2,1,629494,630387
3,1,633523,634601
4,1,778319,779215
...,...,...,...
67145,Y,56867424,56868313
67146,Y,56869540,56870326
67147,Y,56870767,56871793
67148,Y,56873588,56874343


Find overlaps between regions (start_b and end_b are from genes)

In [17]:
# Find overlaps
overlaps = peaks_pr.join(genes_pr)

# Convert overlaps to DataFrame for easier manipulation
overlap_df = overlaps.df
overlap_df

,Chromosome,Start,End,Start_b,End_b,gene_id
0,1,181027,181582,89295,181719,ENSG00000241860
1,1,181027,181582,146386,199861,ENSG00000310528
2,1,181027,181582,180931,184937,ENSG00000308415
3,1,629494,630387,586054,633129,ENSG00000293331
4,1,629494,630387,586945,827989,ENSG00000225880
...,...,...,...,...,...,...
79065,Y,11314279,11314998,11314921,11316187,ENSG00000259154
79066,Y,11315914,11316892,11314921,11316187,ENSG00000259154
79067,Y,11322299,11323004,11321557,11322823,ENSG00000259029
79068,Y,11332807,11333567,11332329,11333595,ENSG00000258991


Gene regions that are fully contained in the peak regions

In [18]:
# Find genes fully contained within peaks
contained_overlaps = peaks_pr.join(genes_pr, how="containment")

# Convert the results to a DataFrame for easier manipulation
contained_df = contained_overlaps.df

# Display the resulting DataFrame
contained_df


,Chromosome,Start,End,Start_b,End_b,gene_id
0,1,181027,181582,89295,181719,ENSG00000241860
1,1,181027,181582,146386,199861,ENSG00000310528
2,1,181027,181582,180931,184937,ENSG00000308415
3,1,629494,630387,586054,633129,ENSG00000293331
4,1,629494,630387,586945,827989,ENSG00000225880
...,...,...,...,...,...,...
57396,X,155070872,155071779,155064034,155147937,ENSG00000214827
57397,X,155216046,155216880,155197007,155239841,ENSG00000155959
57398,X,155612416,155613272,155489011,155719098,ENSG00000185973
57399,X,155767128,155768009,155612572,155782459,ENSG00000168939


# Preprocessing

In [79]:
row = contained_df[contained_df['Start'] == 29553]
print(row)

Empty DataFrame
Columns: [Chromosome, Start, End, Start_b, End_b, gene_id]
Index: []


In [66]:
from muon import atac as ac

In [72]:
dat= ac.tl.get_gene_annotation_from_rna(mdata['rna']).head(3)  # accepts MuData with 'rna' modality or mdata['rna'] AnnData directly

In [73]:
dat


,Chromosome,Start,End,gene_id,gene_name
MIR1302-2HG,chr1,29553,30267,ENSG00000243485,MIR1302-2HG
FAM138A,chr1,36080,36081,ENSG00000237613,FAM138A
OR4F5,chr1,65418,69055,ENSG00000186092,OR4F5


In [76]:
row = dat[dat['Start'] == 29553]
print(row)


            Chromosome  Start    End          gene_id    gene_name
MIR1302-2HG       chr1  29553  30267  ENSG00000243485  MIR1302-2HG


In [95]:
rna = mdata.mod['rna']
rna  # an AnnData object

AnnData object with n_obs × n_vars = 22772 × 36601
    var: 'gene_ids', 'feature_types', 'genome', 'interval'

In [96]:
rna.var

,gene_ids,feature_types,genome,interval
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38,chr1:29553-30267
FAM138A,ENSG00000237613,Gene Expression,GRCh38,chr1:36080-36081
OR4F5,ENSG00000186092,Gene Expression,GRCh38,chr1:65418-69055
AL627309.1,ENSG00000238009,Gene Expression,GRCh38,chr1:120931-133723
AL627309.3,ENSG00000239945,Gene Expression,GRCh38,chr1:91104-91105
...,...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38,KI270728.1:1270983-1270984
AC023491.2,ENSG00000278633,Gene Expression,GRCh38,KI270731.1:13000-13001
AC007325.1,ENSG00000276017,Gene Expression,GRCh38,KI270734.1:72410-72411
AC007325.4,ENSG00000278817,Gene Expression,GRCh38,KI270734.1:131493-131494
